In [25]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [26]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [27]:
# Read data (pdf) files
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,ServiceContext
documents=SimpleDirectoryReader("data/pdf").load_data()

In [28]:
#documents

In [29]:
# Convert this documents to index
service_context = ServiceContext.from_defaults(chunk_size=512)
index=VectorStoreIndex.from_documents(documents, service_context=service_context ,show_progress=True)
index

/var/folders/tt/s83h4n255y95sgyr2yrtv2x00000gp/T/ipykernel_16937/1848985020.py:2: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(chunk_size=512)


Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# Get the Query Engine from the Index
query_engine = index.as_query_engine()
query_engine

In [19]:
# Ask a query to this Query Engine
response = query_engine.query("Tell me something about Ramesh")
print(response)

Ramesh is from Odisha and spent his childhood in Bhubaneswar. He completed his schooling from Central School and intermediate from BJB Junior college. He pursued his engineering (B.Tech) from ITER, Bhubaneswar. After graduation, he worked as a Systems Engineer at TCS for 5 years before moving to HP for a brief period of 7 months. Currently, he is working at Oracle as a Technical Lead for the past 4 years.


In [20]:
# See the responses (Top response along with top - x responses)
from llama_index.core.response.pprint_utils import pprint_response
pprint_response(response, show_source=True)

Final Response: Ramesh is from Odisha and spent his childhood in
Bhubaneswar. He completed his schooling from Central School and
intermediate from BJB Junior college. He pursued his engineering
(B.Tech) from ITER, Bhubaneswar. After graduation, he worked as a
Systems Engineer at TCS for 5 years before moving to HP for a brief
period of 7 months. Currently, he is working at Oracle as a Technical
Lead for the past 4 years.
______________________________________________________________________
Source Node 1/2
Node ID: d58b3877-c009-4c81-8a05-addde85b2d18
Similarity: 0.8567114642495725
Text: Ramesh belongs to Odisha. He spent his childhood in Bhubaneswar
– the capital of Odisha. He did his schooling from Central School and
completed his intermediate from BJB Junior college. Post that he did
his engineering (B.Tech) from ITER, Bhubaneswar.   AFer graduaGon, he
joined TCS as a Systems Engineer where he served for 5 years. Then he
swit...
______________________________________________________

## Create a query engine using Vector Retriever

In [21]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.indices.postprocessor import SimilarityPostprocessor

retriever = VectorIndexRetriever(index=index, similarity_top_k=4)
post_processor = SimilarityPostprocessor(similarity_cutoff=0.80)
query_engine = RetrieverQueryEngine(retriever=retriever, node_postprocessors=[post_processor])


In [22]:
# Ask a query to this Query Engine
response = query_engine.query("Tell me something about Ramesh")
# See the responses (Top response along with top - 4 responses)
pprint_response(response, show_source=True)

Final Response: Ramesh is from Odisha and spent his childhood in
Bhubaneswar. He completed his schooling from Central School and his
intermediate education from BJB Junior College. He pursued his
engineering (B.Tech) from ITER, Bhubaneswar. After graduation, he
started his career as a Systems Engineer at TCS, then moved to HP for
a brief period before joining Oracle, where he currently serves as a
Technical Lead.
______________________________________________________________________
Source Node 1/1
Node ID: d58b3877-c009-4c81-8a05-addde85b2d18
Similarity: 0.8567114642495725
Text: Ramesh belongs to Odisha. He spent his childhood in Bhubaneswar
– the capital of Odisha. He did his schooling from Central School and
completed his intermediate from BJB Junior college. Post that he did
his engineering (B.Tech) from ITER, Bhubaneswar.   AFer graduaGon, he
joined TCS as a Systems Engineer where he served for 5 years. Then he
swit...


# Everything in one place

In [30]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage
)

INDEX_STORE = './index_storage'

if not os.path.exists(INDEX_STORE):
    # Load the documents, create the index and store it for later usage
    documents = SimpleDirectoryReader("data/pdf").load_data()
    index = VectorStoreIndex.from_documents(documents,show_progress=True)
    # Store the index
    index.storage_context.persist(persist_dir=INDEX_STORE)
else:
    # Load the index from file
    storage_context = StorageContext.from_defaults(persist_dir=INDEX_STORE)
    index = load_index_from_storage(storage_context=storage_context)


query_engine = index.as_query_engine()
response = query_engine.query("How many companies Ramesh has worked so far ?");
print(response)
response = query_engine.query("Does Ramesh belong to Africa ? Show the reason ");
print(response)

Parsing nodes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Ramesh has worked in three companies so far.
No, Ramesh does not belong to Africa. The reason is that he spent his childhood in Bhubaneswar, the capital of Odisha, and completed his education in Odisha as well.
